In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json
from IPython.core.display import display, HTML
import tempfile

from math import sqrt
import scipy.stats
import sys

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as px

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import plot_tree

import xgboost as xgb

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
df_results=pd.read_csv("/content/drive/MyDrive/2022/BigData/results.csv")
df_results.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
5,1876-03-25,Scotland,Wales,4.0,0.0,Friendly,Glasgow,Scotland,False
6,1877-03-03,England,Scotland,1.0,3.0,Friendly,London,England,False
7,1877-03-05,Wales,Scotland,0.0,2.0,Friendly,Wrexham,Wales,False
8,1878-03-02,Scotland,England,7.0,2.0,Friendly,Glasgow,Scotland,False
9,1878-03-23,Scotland,Wales,9.0,0.0,Friendly,Glasgow,Scotland,False


In [ ]:
df_shootouts=pd.read_csv("/content/drive/MyDrive/2022/BigData/shootouts.csv")

In [ ]:
df_player15=pd.read_excel("/content/drive/MyDrive/2022/BigData/Career Mode player datasets - FIFA 15-22.xlsx", sheet_name=0)

In [ ]:
df_results15=df_results.loc[df_results['date']>='2015-01-01'].loc[df_results['date']<'2016-01-01']
df_results15.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
36970,2015-01-04,Bahrain,Jordan,1.0,0.0,Friendly,Ballarat,Australia,True
36971,2015-01-04,Iran,Iraq,1.0,0.0,Friendly,Wollongong,Australia,True
36972,2015-01-04,South Korea,Saudi Arabia,2.0,0.0,Friendly,Parramatta,Australia,True
36973,2015-01-04,South Africa,Zambia,1.0,0.0,Friendly,Johannesburg,South Africa,False
36974,2015-01-05,China PR,Oman,4.0,1.0,Friendly,Penrith,Australia,True
36975,2015-01-07,Cameroon,DR Congo,1.0,1.0,Friendly,Yaoundé,Cameroon,False
36976,2015-01-09,Australia,Kuwait,4.0,1.0,AFC Asian Cup,Melbourne,Australia,False
36977,2015-01-09,Senegal,Gabon,1.0,0.0,Friendly,Casablanca,Morocco,True
36978,2015-01-10,Burkina Faso,Eswatini,5.0,1.0,Friendly,Nelspruit,South Africa,True
36979,2015-01-10,Cameroon,South Africa,1.0,1.0,Friendly,Libreville,Gabon,True


In [ ]:
df_player15['rank_in_nation'] = df_player15.groupby("nationality_id")['value_eur'].rank(method="first", ascending=False)
df_player15['rank_in_nation'] = df_player15['rank_in_nation'].astype('Int64')
df_player15.sort_values('value_eur', ascending=False)

In [ ]:
df_player15['rank_in_nation'] = df_player15.groupby("nationality_name")['value_eur'].rank(method="first", ascending=False)
df_player15['rank_in_nation'] = df_player15['rank_in_nation'].astype('Int64')
df_player15.sort_values('value_eur', ascending=False)

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,rank_in_nation
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,27,...,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,1
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,29,...,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,1
4,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,300000.0,28,...,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,1
2,9014,https://sofifa.com/player/9014/arjen-robben/15...,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,...,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/105035/60.png,https://cdn.sofifa.net/flags/nl.png,1
3,41236,https://sofifa.com/player/41236/zlatan-ibrahim...,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,32,...,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1363/60.png,https://cdn.sofifa.net/flags/se.png,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15200,174441,https://sofifa.com/player/174441/rocus-lamare/...,R. Lamare,Rocus Lamare,CM,52,55,NaN,NaN,27,...,52,52,53,13,https://cdn.sofifa.net/players/174/441/15_120.png,NaN,NaN,https://cdn.sofifa.net/teams/111466/60.png,https://cdn.sofifa.net/flags/in.png,<NA>
15452,225444,https://sofifa.com/player/225444/joel-stevens/...,J. Stevens,Joel Craig Stevens,"RM, CAM",52,56,NaN,NaN,19,...,35,35,37,10,https://cdn.sofifa.net/players/225/444/15_120.png,NaN,NaN,https://cdn.sofifa.net/teams/111473/60.png,https://cdn.sofifa.net/flags/nz.png,<NA>
15534,216751,https://sofifa.com/player/216751/diego-bejaran...,D. Bejarano,Diego Bejarano Ibañez,"RB, CB, RM",51,54,NaN,NaN,22,...,52,52,51,14,https://cdn.sofifa.net/players/216/751/15_120.png,NaN,NaN,https://cdn.sofifa.net/teams/111451/60.png,https://cdn.sofifa.net/flags/bo.png,<NA>
15806,223760,https://sofifa.com/player/223760/amrinder-sing...,A. Singh,Amrinder Singh,GK,50,54,NaN,NaN,21,...,31,31,28,50,https://cdn.sofifa.net/players/223/760/15_120.png,NaN,NaN,https://cdn.sofifa.net/teams/111466/60.png,https://cdn.sofifa.net/flags/in.png,<NA>


In [ ]:
# 'sofifa_id','short_name','long_name','vallue_eur','wage_eur','age','dob','nationality_id','nationality_name','nation_team_id','nation_position','nation_jersey_number','preferred_foot','weak_foot','skill_moves','international_reputation','pace'

df_player15_edited = pd.DataFrame(columns=['nations_id', 'player1_value_eur', 'player1_wage_eur'])

for i in range(2,12):
    col1 = 'player' + str(i) + '_value_eur'
    col2 = 'player' + str(i) + '_wage_eur'
    df_player15_edited[col1] = np.nan
    df_player15_edited[col2] = np.nan

print(df_player15_edited)

Empty DataFrame
Columns: [nations_id, player1_value_eur, player1_wage_eur, player2_value_eur, player2_wage_eur, player3_value_eur, player3_wage_eur, player4_value_eur, player4_wage_eur, player5_value_eur, player5_wage_eur, player6_value_eur, player6_wage_eur, player7_value_eur, player7_wage_eur, player8_value_eur, player8_wage_eur, player9_value_eur, player9_wage_eur, player10_value_eur, player10_wage_eur, player11_value_eur, player11_wage_eur]
Index: []

[0 rows x 23 columns]


In [ ]:
df_player15.iloc[0]['sofifa_id']

158023

In [ ]:
nationality_ids = df_player15['nationality_id'].tolist()

In [ ]:
for i in nationality_ids:
    player = []
    for j in range(1, 12):
        player.append(df_player15.loc[df_player15['rank_in_nation'] == j].loc[df_player15['nationality_id'] == 52]['value_eur'].item())
        player.append(df_player15.loc[df_player15['rank_in_nation']==j].loc[df_player15['nationality_id']== 52]['wage_eur'].item())
    print(player)
    df_player15_edited.loc[len(df_player15_edited.index)] = player

df_player15_edited

[100500000.0, 550000.0, 45500000.0, 230000.0, 45500000.0, 230000.0, 27000000.0, 170000.0, 21000000.0, 160000.0, 18000000.0, 140000.0, 14500000.0, 150000.0, 14500000.0, 140000.0, 13000000.0, 130000.0, 12500000.0, 130000.0, 12500000.0, 90000.0]


ValueError: ignored

In [ ]:
df_player15.loc[df_player15['rank_in_nation']==1].loc[df_player15['nationality_id'] == 52]['value_eur'].iloc[0]

100500000.0

In [ ]:
player = [52,]
for j in range(1, 12):
    player.append(df_player15.loc[df_player15['rank_in_nation'] == j].loc[df_player15['nationality_id'] == 52]['value_eur'].item())
    player.append(df_player15.loc[df_player15['rank_in_nation']==j].loc[df_player15['nationality_id']== 52]['wage_eur'].item())
print(player)
df_player15_edited.loc[len(df_player15_edited.index)] = player


[52, 100500000.0, 550000.0, 45500000.0, 230000.0, 45500000.0, 230000.0, 27000000.0, 170000.0, 21000000.0, 160000.0, 18000000.0, 140000.0, 14500000.0, 150000.0, 14500000.0, 140000.0, 13000000.0, 130000.0, 12500000.0, 130000.0, 12500000.0, 90000.0]


In [ ]:
df_player15.loc[df_player15['rank_in_nation'] == 2].loc[df_player15['nationality_id'] == 52]['value_eur']

23    45500000.0
26    45500000.0
Name: value_eur, dtype: float64

In [ ]:
# home_score away_score home_player1_wage home_player2_wage home_player3_wage ... home_player11_wage away_player1_wage ... away_player11_wage